# Custom vs. Scikit learn Built Decision Tree


## Buld a Custom Decision Tree


In [1]:
import numpy as np

class CustomDecisionTree:
    def __init__(self, max_depth=None):
        """
        Initializes the decision tree with the specified maximum depth.

        Parameters:
        max_depth (int, optional): The maximum depth of the tree. If None, the tree is expanded until all leaves are pure or contain fewer than the minimum samples required to split.
        """
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        """
        Trains the decision tree model using the provided training data.

        Parameters:
        X (array-like): Feature matrix (n_samples, n_features) for training the model.
        y (array-like): Target labels (n_samples,) for training the model.
        """
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        """
        Recursively builds the decision tree by splitting the data based on the best feature and threshold.

        Parameters:
        X (array-like): Feature matrix (n_samples, n_features) for splitting.
        y (array-like): Target labels (n_samples,) for splitting.
        depth (int, optional): Current depth of the tree during recursion.

        Returns:
        dict: A dictionary representing the structure of the tree, containing the best feature index, threshold, and recursive tree nodes.
        """
        num_samples, num_features = X.shape
        unique_classes = np.unique(y)

        if len(unique_classes) == 1:
            return {'class': unique_classes[0]}
        if num_samples == 0 or (self.max_depth and depth >= self.max_depth):
            return {'class': np.bincount(y).argmax()}

        best_info_gain = -float('inf')
        best_split = None
        for feature_idx in range(num_features):
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                left_mask = X[:, feature_idx] <= threshold
                right_mask = ~left_mask
                left_y = y[left_mask]
                right_y = y[right_mask]

                info_gain = self._information_gain(y, left_y, right_y)

                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    best_split = {
                        'feature_idx': feature_idx,
                        'threshold': threshold,
                        'left_y': left_y,
                        'right_y': right_y,
                    }

        if best_split is None:
            return {'class': np.bincount(y).argmax()}

        left_tree = self._build_tree(X[best_split['left_y']], best_split['left_y'], depth + 1)
        right_tree = self._build_tree(X[best_split['right_y']], best_split['right_y'], depth + 1)

        return {'feature_idx': best_split['feature_idx'], 'threshold': best_split['threshold'],
                'left_tree': left_tree, 'right_tree': right_tree}

    def _information_gain(self, parent, left, right):
        """
        Computes the Information Gain between the parent node and the left/right child nodes.

        Parameters:
        parent (array-like): The labels of the parent node.
        left (array-like): The labels of the left child node.
        right (array-like): The labels of the right child node.

        Returns:
        float: The Information Gain of the split.
        """
        parent_entropy = self._entropy(parent)
        left_entropy = self._entropy(left)
        right_entropy = self._entropy(right)

        weighted_avg_entropy = (len(left) / len(parent)) * left_entropy + (len(right) / len(parent)) * right_entropy
        return parent_entropy - weighted_avg_entropy

    def _entropy(self, y):
        """
        Computes the entropy of a set of labels.

        Parameters:
        y (array-like): The labels for which entropy is calculated.

        Returns:
        float: The entropy of the labels.
        """
        class_probs = np.bincount(y) / len(y)avoid log(0)
        return -np.sum(class_probs * np.log2(class_probs + 1e-9))

    def predict(self, X):
        """
        Predicts the target labels for the given test data based on the trained decision tree.

        Parameters:
        X (array-like): Feature matrix (n_samples, n_features) for prediction.

        Returns:
        list: A list of predicted target labels (n_samples,).
        """
        return [self._predict_single(x, self.tree) for x in X]

    def _predict_single(self, x, tree):
        """
        Recursively predicts the target label for a single sample by traversing the tree.

        Parameters:
        x (array-like): A single feature vector for prediction.
        tree (dict): The current subtree or node to evaluate.

        Returns:
        int: The predicted class label for the sample.
        """
        if 'class' in tree:
            return tree['class']

        feature_val = x[tree['feature_idx']]
        if feature_val <= tree['threshold']:
            return self._predict_single(x, tree['left_tree'])
        else:
            return self._predict_single(x, tree['right_tree'])


## Necessary Imports


In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

## Load and Split the Iris Dataset


In [3]:
# Load the Iris dataset
data = load_wine()
X = data.data
y = data.target

# Split into training and test sets (80% training, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Train and Evaluate the Custom Decision Tree


In [4]:
# Train the custom decision tree
custom_tree = CustomDecisionTree(max_depth=3)
custom_tree.fit(X_train, y_train)

# Predict on the test set
y_pred_custom = custom_tree.predict(X_test)

# Calculate accuracy
accuracy_custom = accuracy_score(y_test, y_pred_custom)
print(f"Custom Decision Tree Accuracy: {accuracy_custom:.4f}")


Custom Decision Tree Accuracy: 0.7222


## Train and Evaluate the scikit-learn Decision Tree


In [5]:
# Train the Scikit-learn decision tree
sklearn_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
sklearn_tree.fit(X_train, y_train)

# Predict on the test set
y_pred_sklearn = sklearn_tree.predict(X_test)

# Calculate accuracy
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)
print(f"Scikit-learn Decision Tree Accuracy: {accuracy_sklearn:.4f}")


Scikit-learn Decision Tree Accuracy: 0.9444


## Result Comparision


In [6]:
print(f"Accuracy Comparison:")
print(f"Custom Decision Tree: {accuracy_custom:.4f}")
print(f"Scikit-learn Decision Tree: {accuracy_sklearn:.4f}")


Accuracy Comparison:
Custom Decision Tree: 0.7222
Scikit-learn Decision Tree: 0.9444


# Random Forest


In [8]:
from sklearn.ensemble import RandomForestClassifier

In [9]:
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred_rf=rf.predict(X_test)

In [10]:
#for f1 score
from sklearn.metrics import f1_score
f1_score_rf=f1_score(y_test,y_pred_rf,average='weighted')
print(f"F1 Score for Random Forest: {f1_score_rf:.4f}")

F1 Score for Random Forest: 1.0000


In [11]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10]
}

In [13]:
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid)
grid_search.fit(X_train, y_train)
print(f"Best Hyperparameters: {grid_search.best_params_}")

Best Hyperparameters: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 100}


In [14]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

In [16]:
y_regression = data.target
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y_regression, test_size=0.3, random_state=42)

In [17]:
dt_regressor = DecisionTreeRegressor(random_state=42)
dt_regressor.fit(X_train_reg, y_train_reg)

rf_regressor = RandomForestRegressor(random_state=42)
rf_regressor.fit(X_train_reg, y_train_reg)

y_pred_dt_reg = dt_regressor.predict(X_test_reg)
y_pred_rf_reg = rf_regressor.predict(X_test_reg)

In [19]:
from sklearn.metrics import mean_squared_error
mse_dt = mean_squared_error(y_test_reg, y_pred_dt_reg)
mse_rf = mean_squared_error(y_test_reg, y_pred_rf_reg)
print(f"Mean Squared Error for Decision Tree Regressor: {mse_dt}")
print(f"Mean Squared Error for Random Forest Regressor: {mse_rf}")

Mean Squared Error for Decision Tree Regressor: 0.14814814814814814
Mean Squared Error for Random Forest Regressor: 0.057238888888888916


In [21]:
param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10]
}
rf_regressor = RandomForestRegressor(random_state=42)
random_search = RandomizedSearchCV(estimator=rf_regressor, param_distributions=param_dist)
random_search.fit(X_train_reg, y_train_reg)
print(f"Best Hyperparameters for Random Forest Regressor: {random_search.best_params_}")

Best Hyperparameters for Random Forest Regressor: {'n_estimators': 300, 'min_samples_split': 5, 'max_depth': 20}
